In [109]:
#Cragamos librerias
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [110]:
#Cargar desde un archivo .csv sin indice
df = pd.read_csv('Datos_limpios_México.csv')
df.head(5)

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,2010-06-28,...,1.0,1.0,7,363,0.0,4.80,4.90,4.90,4.90,1.70
1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,2010-08-09,...,1.0,1.0,365,325,64.0,4.58,4.70,4.78,4.98,0.41
2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,2011-01-04,...,2.0,2.0,30,325,49.0,4.90,4.75,4.92,4.98,0.31
3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,2010-08-24,...,1.0,1.6,180,257,127.0,4.91,4.96,4.98,4.96,0.83
4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,2011-04-27,...,1.0,1.0,365,275,11.0,4.91,5.00,4.73,4.91,0.11


**Convertimos a numericos**

In [111]:
#df['host_response_time'] = df['host_response_time'].map({'a few days or more':1, 'within an hour':2,'within a few hours':3, 'within a day':4})
df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1, 'f': 0})
df["host_response_rate"] = df["host_response_rate"].str.rstrip('%').astype(float) / 100
#df['room_type'] = df['room_type'].map({'Entire home/apt': 1,'Private room': 2,'Shared room': 3,'Hotel room': 4})
#df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100
df['host_verifications'] = df['host_verifications'].map({"['email', 'phone', 'work_email']":1, "['email', 'phone']":2, "['phone']": 3,"['phone', 'work_email']": 4, "['email']": 5,"[]": 6,"['email', 'work_email']": 7,"['work_email']": 8})
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': 1, 'f': 0})
df['host_identity_verified'] = df['host_identity_verified'].map({'t': 1, 'f': 0})
df['has_availability'] = df['has_availability'].map({'t': 1, 'f': 0})
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})


In [112]:
df['property_type'] = df['property_type'].replace({
    'Entire villa': 1, 'Entire home': 2, 'Entire rental unit': 3, 'Entire loft': 4,
    'Private room': 5, 'Private room in rental unit': 6, 'Private room in home': 7,
    'Private room in condo': 8, 'Private room in guest suite': 9, 'Entire condo': 10,
    'Private room in serviced apartment': 11, 'Room in serviced apartment': 12,
    'Entire guest suite': 13, 'Private room in townhouse': 14, 'Room in boutique hotel': 15,
    'Private room in bed and breakfast': 16, 'Shared room in rental unit': 17,
    'Entire serviced apartment': 18, 'Tiny home': 19, 'Entire guesthouse': 20,
    'Private room in guesthouse': 21, 'Private room in loft': 22, 'Entire bungalow': 23,
    'Casa particular': 24, 'Shared room in loft': 25, 'Entire townhouse': 26,
    'Private room in tiny home': 27, 'Shared room in guesthouse': 28, 'Room in bed and breakfast': 29,
    'Shared room in hostel': 30, 'Private room in farm stay': 31, 'Shared room in home': 32,
    'Private room in villa': 33, 'Entire cabin': 34, 'Private room in casa particular': 35,
    'Room in hotel': 36, 'Private room in floor': 37, 'Private room in cabin': 38,
    'Shared room in guest suite': 39, 'Room in aparthotel': 40, 'Entire place': 41,
    'Private room in hostel': 42, 'Earthen home': 43, 'Entire hostel': 44,
    'Private room in earthen home': 45, 'Private room in dorm': 46, 'Private room in hut': 47,
    'Entire cottage': 48, 'Entire chalet': 49, 'Shared room in dorm': 50,
    'Entire in-law': 51, 'Holiday park': 52, 'Private room in dome': 53,
    'Shared room in condo': 54, 'Campsite': 55, 'Private room in vacation home': 56,
    'Room in casa particular': 57, 'Castle': 58, 'Private room in barn': 59,
    'Shared room in hotel': 60, 'Farm stay': 61, 'Shared room in tiny home': 62,
    'Hut': 63, 'Shared room in cabin': 64, 'Private room in cottage': 65,
    'Entire home/apt': 66, 'Room in hostel': 67, 'Shared room in serviced apartment': 68,
    'Entire vacation home': 69, 'Shared room in tent': 70, 'Shared room in bed and breakfast': 71,
    'Private room in shipping container': 72, 'Private room in nature lodge': 73,
    'Private room in tower': 74, 'Private room in lighthouse': 75,
    'Shared room in casa particular': 76, 'Shared room in townhouse': 77,
    'Shared room in farm stay': 78, 'Shared room in boutique hotel': 79,
    'Shipping container': 80, 'Private room in pension': 81, 'Private room in bungalow': 82,
    'Dome': 83, 'Tent': 84, 'Private room in chalet': 85, 'Private room in tent': 86,
    'Private room in castle': 87, 'Private room in houseboat': 88, 'Boat': 89,
    'Private room in resort': 90, 'Private room in minsu': 91, 'Tower': 92
})


C:\Users\kidfr\AppData\Local\Temp\ipykernel_23744\2798425177.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['property_type'] = df['property_type'].replace({


**Reemplazar valores**

_host_acceptance_rate_

In [113]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_acceptance_rate'])
unico

array(['0%', '0.9', '10%', '100%', '11%', '13%', '14%', '15%', '17%',
       '18%', '19%', '2%', '20%', '21%', '22%', '23%', '24%', '25%',
       '26%', '27%', '28%', '29%', '3%', '30%', '31%', '32%', '33%',
       '34%', '35%', '36%', '37%', '38%', '39%', '4%', '40%', '41%',
       '42%', '43%', '44%', '45%', '46%', '47%', '48%', '49%', '5%',
       '50%', '51%', '52%', '53%', '54%', '55%', '56%', '57%', '58%',
       '59%', '6%', '60%', '61%', '62%', '63%', '64%', '65%', '66%',
       '67%', '68%', '69%', '7%', '70%', '71%', '72%', '73%', '74%',
       '75%', '76%', '77%', '78%', '79%', '8%', '80%', '81%', '82%',
       '83%', '84%', '85%', '86%', '87%', '88%', '89%', '9%', '90%',
       '91%', '92%', '93%', '94%', '95%', '96%', '97%', '98%', '99%'],
      dtype=object)

In [114]:
df["host_acceptance_rate"] = df["host_acceptance_rate"].str.rstrip('%').astype(float) / 100

In [115]:
#Convierto una variable a dicotómica
df['host_acceptance_rate'] = df['host_acceptance_rate'].apply(
    lambda x: 1 if isinstance(x, (int, float)) and x > 0.80 else (0 if isinstance(x, (int, float)) and x <= 0.80 else 'Sin respuesta')
)

df

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,2010-06-28,...,1.0,1.0,7,363,0.0,4.80,4.90,4.90,4.90,1.70
1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,2010-08-09,...,1.0,1.0,365,325,64.0,4.58,4.70,4.78,4.98,0.41
2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,2011-01-04,...,2.0,2.0,30,325,49.0,4.90,4.75,4.92,4.98,0.31
3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,2010-08-24,...,1.0,1.6,180,257,127.0,4.91,4.96,4.98,4.96,0.83
4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,2011-04-27,...,1.0,1.0,365,275,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,2016-04-16,...,2.0,1.0,45,365,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70


_room_type_

In [116]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [117]:
#Convierto una variable a dicotómica
df['room_type'] = df['room_type'].replace(["Shared room", "Private room"], 0)
df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)
df

C:\Users\kidfr\AppData\Local\Temp\ipykernel_23744\508382164.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['room_type'] = df['room_type'].replace(["Entire home/apt", "Hotel room"], 1)


,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,2010-06-28,...,1.0,1.0,7,363,0.0,4.80,4.90,4.90,4.90,1.70
1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,2010-08-09,...,1.0,1.0,365,325,64.0,4.58,4.70,4.78,4.98,0.41
2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,2011-01-04,...,2.0,2.0,30,325,49.0,4.90,4.75,4.92,4.98,0.31
3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,2010-08-24,...,1.0,1.6,180,257,127.0,4.91,4.96,4.98,4.96,0.83
4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,2011-04-27,...,1.0,1.0,365,275,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,2016-04-16,...,2.0,1.0,45,365,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,177,0.0,4.80,4.90,4.90,4.90,1.70


_availability_365_

In [118]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['availability_365'])
unico

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [119]:
#Convierto una variable a dicotómica
df['availability_365'] = df['availability_365'].apply(
    lambda x: 1 if x == 365 else (0 if isinstance(x, (int, float)) and x < 365 else 'Sin respuesta')
)

df

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,2010-06-28,...,1.0,1.0,7,0,0.0,4.80,4.90,4.90,4.90,1.70
1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,2010-08-09,...,1.0,1.0,365,0,64.0,4.58,4.70,4.78,4.98,0.41
2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,2011-01-04,...,2.0,2.0,30,0,49.0,4.90,4.75,4.92,4.98,0.31
3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,2010-08-24,...,1.0,1.6,180,0,127.0,4.91,4.96,4.98,4.96,0.83
4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,2011-04-27,...,1.0,1.0,365,0,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,2016-04-16,...,2.0,1.0,45,1,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70


_host_response_time_

In [120]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['host_response_time'])
unico

array(['a few days or more', 'within a day', 'within a few hours',
       'within an hour'], dtype=object)

In [121]:
df['host_response_time']=df['host_response_time'].replace(["within a few hours", "within a day", "a few days or more"], 0)
df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)
df

C:\Users\kidfr\AppData\Local\Temp\ipykernel_23744\1826624729.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['host_response_time']=df['host_response_time'].replace(['within an hour'], 1)


,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,2010-06-28,...,1.0,1.0,7,0,0.0,4.80,4.90,4.90,4.90,1.70
1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,2010-08-09,...,1.0,1.0,365,0,64.0,4.58,4.70,4.78,4.98,0.41
2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,2011-01-04,...,2.0,2.0,30,0,49.0,4.90,4.75,4.92,4.98,0.31
3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,2010-08-24,...,1.0,1.6,180,0,127.0,4.91,4.96,4.98,4.96,0.83
4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,2011-04-27,...,1.0,1.0,365,0,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,2016-04-16,...,2.0,1.0,45,1,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70


_accommodates_

In [122]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['accommodates'])
unico

array([1. , 2. , 2.9, 3. , 4. , 5. , 6. , 7. ])

In [123]:
df['accommodates'] = df['accommodates'].replace([1, 2, 2.9], 0).replace([3, 4, 5, 6, 7], 1)
df

,Unnamed: 0,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_url,host_name,host_since,...,beds,minimum_nights,maximum_nights,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_communication,review_scores_location,reviews_per_month
0,0,https://www.airbnb.com/rooms/35797,2024-09-25,city scrape,Villa Dante,"Santa Fe Mall, Interloma Park and the Lion Des...",https://a0.muscache.com/pictures/f395ab78-1185...,https://www.airbnb.com/users/show/153786,Dici,2010-06-28,...,1.0,1.0,7,0,0.0,4.80,4.90,4.90,4.90,1.70
1,1,https://www.airbnb.com/rooms/44616,2024-09-26,previous scrape,Condesa Haus,The neighborhood is very nice,https://a0.muscache.com/pictures/251410/ec75fe...,https://www.airbnb.com/users/show/196253,Fernando,2010-08-09,...,1.0,1.0,365,0,64.0,4.58,4.70,4.78,4.98,0.41
2,2,https://www.airbnb.com/rooms/67703,2024-09-26,city scrape,"2 bedroom apt. deco bldg, Condesa",The neighborhood is very nice,https://a0.muscache.com/pictures/3281720/6f078...,https://www.airbnb.com/users/show/334451,Nicholas,2011-01-04,...,2.0,2.0,30,0,49.0,4.90,4.75,4.92,4.98,0.31
3,3,https://www.airbnb.com/rooms/70644,2024-09-25,city scrape,Beautiful light Studio Coyoacan- full equipped !,Coyoacan is a beautiful neighborhood famous fo...,https://a0.muscache.com/pictures/f397d2da-d045...,https://www.airbnb.com/users/show/212109,Trisha,2010-08-24,...,1.0,1.6,180,0,127.0,4.91,4.96,4.98,4.96,0.83
4,4,https://www.airbnb.com/rooms/107078,2024-09-25,city scrape,NEW DESIGNER LOFT,"Is located in the best area of Mexico City, Po...",https://a0.muscache.com/pictures/bd64f86d-4931...,https://www.airbnb.com/users/show/540705,Andrea,2011-04-27,...,1.0,1.0,365,0,11.0,4.91,5.00,4.73,4.91,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26577,26577,https://www.airbnb.com/rooms/1252141718111439700,2024-09-25,city scrape,Charming 2-Bedroom Retreat in Lomas de Bezares,"Welcome to Lomas de Bezares, a neighborhood kn...",https://a0.muscache.com/pictures/prohost-api/H...,https://www.airbnb.com/users/show/67472234,HostPal,2016-04-16,...,2.0,1.0,45,1,0.0,4.80,4.90,4.90,4.90,1.70
26578,26578,https://www.airbnb.com/rooms/1252159821531694005,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70
26579,26579,https://www.airbnb.com/rooms/1252185591721328995,2024-09-25,city scrape,Suite King | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70
26580,26580,https://www.airbnb.com/rooms/1252218347521498445,2024-09-25,city scrape,Suite Queen | District Polanco,The neighborhood is very nice,https://a0.muscache.com/pictures/miso/Hosting-...,https://www.airbnb.com/users/show/596009321,Luis Angel,2024-08-13,...,1.0,1.0,365,0,0.0,4.80,4.90,4.90,4.90,1.70


In [124]:
df['availability_365']

0        0
1        0
2        0
3        0
4        0
        ..
26577    1
26578    0
26579    0
26580    0
26581    0
Name: availability_365, Length: 26582, dtype: int64

**REGRESIÓN LOGÍSTICA**

_host_is_superhost_

In [125]:
#Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_acceptance_rate', 'host_response_rate', 'number_of_reviews']]
Var_Dep = df['host_is_superhost']

In [126]:
X= Vars_Indep
y= Var_Dep

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=None)

In [128]:
escalar =StandardScaler()

In [129]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [130]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [131]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [132]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 0, ..., 0, 0, 0], shape=(15950,))

In [133]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[7640 2077]
 [2684 3549]]


In [134]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.6308211873444721


In [135]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.7015047021943573


In [136]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7862509004836884


_host_identity_verified_

In [137]:
#Declaramos las variables dependientes e independientes para regresión logistica
Vars_Indep=df[['has_availability', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['host_identity_verified']

In [138]:
df['host_acceptance_rate'].dtype

dtype('int64')

In [139]:
X1= Vars_Indep
y1= Var_Dep

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=None)

In [141]:
escalar =StandardScaler()

In [142]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [143]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [144]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [145]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(7975,))

In [146]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 225  110]
 [  67 7573]]


In [147]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.985682676038006


In [148]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9778056426332289


In [149]:
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.6716417910447762


_host_acceptance_rate_

In [150]:
Vars_Indep= df[["accommodates", "room_type","instant_bookable"]]
Var_Dep= df["host_acceptance_rate"]

In [151]:
X= Vars_Indep
y= Var_Dep

In [152]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None)

In [153]:
escalar= StandardScaler()

In [154]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [155]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [156]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [157]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(7975,))

In [158]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[   0 1869]
 [   0 6106]]


In [159]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.7656426332288401


In [160]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7656426332288401


In [161]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.0


_has_availability_

In [162]:
Vars_Indep=df[['host_identity_verified', 'host_response_rate', 'host_acceptance_rate']]
Var_Dep=df['has_availability']

In [163]:
df['host_identity_verified'].dtype

dtype('int64')

In [164]:
#Redefinimos las variables
X3 = Vars_Indep
y3 = Var_Dep

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.2, random_state=None)

In [166]:
escalar =StandardScaler()

In [167]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [168]:
from sklearn.linear_model import LogisticRegression
algoritma = LogisticRegression()

In [169]:
algoritma.fit(X_train, y_train)

LogisticRegression()

In [170]:
y_pred =algoritma.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(5317,))

In [171]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 157   63]
 [  13 5084]]


In [172]:
#Calcula la prescisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.987759860112687


In [173]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9857062253150273


In [174]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad =recall_score(y_test, y_pred, average="binary", pos_label= 0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7136363636363636


_instant_bookable_

In [175]:
Vars_Indep= df[["accommodates", "room_type"]]
Var_Dep= df["instant_bookable"]

In [176]:
X= Vars_Indep
y= Var_Dep

In [177]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.4, random_state=None) 

In [178]:
escalar= StandardScaler()

In [179]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [180]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [181]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [182]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(10633,))

In [183]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[6435    0]
 [4198    0]]


In [184]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.0


c:\Users\kidfr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [185]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.6051913853098844


In [186]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_room_type_

In [187]:
Vars_Indep= df[["accommodates", "room_type", "instant_bookable"]]
Var_Dep= df["room_type"]

In [188]:
X= Vars_Indep
y= Var_Dep

In [189]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [190]:
escalar= StandardScaler()

In [191]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [192]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [193]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [194]:
y_pred= algoritmo.predict(X_test)
y_pred

array([0, 1, 1, ..., 1, 0, 1], shape=(7975,))

In [195]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[2668    0]
 [   0 5307]]


In [196]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
1.0


In [197]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
1.0


In [198]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
1.0


_accommodates_

In [199]:
Vars_Indep= df[[ "room_type", "instant_bookable"]]
Var_Dep= df["accommodates"]

In [200]:
X= Vars_Indep
y= Var_Dep

In [201]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=None) 

In [202]:
escalar= StandardScaler()

In [203]:
X_train= escalar.fit_transform(X_train)
X_test= escalar.fit_transform(X_test)

In [204]:
from sklearn.linear_model import LogisticRegression
algoritmo= LogisticRegression()

In [205]:
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [206]:
y_pred= algoritmo.predict(X_test)
y_pred

array([1., 0., 1., ..., 0., 1., 1.], shape=(7975,))

In [207]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print("Matriz de confusion: ")
print(matriz)

Matriz de confusion: 
[[2455 2007]
 [ 327 3186]]


In [208]:
from sklearn.metrics import precision_score

precision= precision_score(y_test, y_pred, average="binary", pos_label=1)
print("predicciones del modelo")
print(precision)

predicciones del modelo
0.6135181975736569


In [209]:
from sklearn.metrics import accuracy_score

exactitud= accuracy_score(y_test, y_pred)
print("Exactitud del modelo")
print(exactitud)

Exactitud del modelo
0.7073354231974922


In [210]:
from sklearn.metrics import recall_score

sensibilidad= recall_score(y_test, y_pred, average="binary", pos_label=0)
print("Sensibilidad del modelo: ")
print(sensibilidad)

Sensibilidad del modelo: 
0.5502017032720753


_availability_365_

In [211]:
df["price"] = df["price"].str.replace("$", "", regex=False).str.replace(",", "").astype(float).astype(int)

In [212]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_response_rate','price', 'review_scores_location', 'property_type', 'host_identity_verified']]
Var_Dep = df['availability_365']

In [213]:
df['review_scores_cleanliness'].dtype

dtype('float64')

In [214]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [215]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [216]:
#Se escalan todos los datos
escalar = StandardScaler()

In [217]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [218]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [219]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [220]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], shape=(7975,))

In [221]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[7573    1]
 [ 401    0]]


In [222]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.0


In [223]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9495924764890282


In [224]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9998679693688935


_host_response_time_

In [225]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'host_total_listings_count', 'host_response_rate']]
Var_Dep = df['host_response_time']

In [226]:
df['host_total_listings_count'].dtype

dtype('float64')

In [227]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [228]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [229]:
#Se escalan todos los datos
escalar = StandardScaler()

In [230]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [231]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [232]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [233]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(7975,))

In [234]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   2 1436]
 [  13 6524]]


In [235]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.8195979899497488


In [236]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.818307210031348


In [237]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.0013908205841446453


_host_has_profile_pic_

In [238]:
#DEclaramos las variables dependientes e independientes para la regresión logística
Vars_Indep=df[['host_verifications', 'review_scores_rating', 'host_total_listings_count', 'host_is_superhost']]
Var_Dep = df['host_has_profile_pic']

In [239]:
df['review_scores_rating'].dtype

dtype('float64')

In [240]:
#Redefinimos las variables
X = Vars_Indep
y = Var_Dep

In [241]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = None)

In [242]:
#Se escalan todos los datos
escalar = StandardScaler()

In [243]:
#Para realizar el escalamineto de las variables "X" tanto de entrenamiento como de prueba, utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [244]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [245]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [246]:
#REalizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], shape=(7975,))

In [247]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   2  150]
 [   7 7816]]


In [248]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.9811699723826262


In [249]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Excatitud del modelo:')
print(exactitud)

Excatitud del modelo:
0.9803134796238244


In [250]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.013157894736842105
